In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta, time
import matplotlib.dates as mdates

In [27]:
#import data
harvest_perrault = pd.read_csv('harvester_perrault.csv')
harvest_pilot = pd.read_csv('harvest_pilot.csv')

how many Perrault Harvester's Operating simultaneously?

In [28]:
harvest_perrault['vpm'] = harvest_perrault['vpm'] * 3 #number of perrault harvesters running per shift

Calculate Productive Time

In [29]:
harvest_perrault['productive_minutes'] = harvest_perrault['strings'] / harvest_perrault['vpm']

In [30]:
perrault = harvest_perrault.groupby('variety')['productive_minutes'].sum().reset_index()

In [31]:
variety_shift_lengths = pd.DataFrame({
    'variety': ['Simcoe', 'Sabro', 'Citra', 'Mosaic', 'HBC 1019', 'Palisade', 'HBC 522', 'Idaho 7', 'Talus', 'HBC 682', 'HBC 638', 'HBC 586', 'Experimental'],
    'length': [timedelta(minutes=600), timedelta(minutes=600), timedelta(minutes=600), timedelta(minutes=600), timedelta(minutes=600), timedelta(minutes=450), timedelta(minutes=450), timedelta(minutes=450), timedelta(minutes=450), timedelta(minutes=450), timedelta(minutes=450), timedelta(minutes=450), timedelta(minutes=450)]
})

In [32]:
# Jeff Finalf Version 
from datetime import datetime, timedelta
import pandas as pd

# Function to check if the given date is a working day (Monday to Saturday)
def is_working_day(date):
    return date.weekday() < 6

# Function to find the next working day starting from current_datetime
def next_working_day(current_datetime):
    next_day = current_datetime + timedelta(days=1)
    while not is_working_day(next_day):
        next_day += timedelta(days=1)
    return next_day

# Function to calculate effective shift time and break time in a shift
def calculate_shift_time_with_breaks(shift_length=timedelta(minutes=600), break_duration=10, lunch_duration=30):
    work_period = timedelta(minutes=240)
    num_breaks_per_shift = shift_length // work_period
    total_break_time_per_shift = num_breaks_per_shift * timedelta(minutes=break_duration)
    effective_shift_time = shift_length - timedelta(minutes=lunch_duration)
    return effective_shift_time, total_break_time_per_shift.total_seconds() / 60  # Convert break time to minutes

# Function to create the shift schedule until all productive minutes are consumed
def create_shift_schedule_with_breaks(data, start_datetime, variety_shift_lengths, break_duration=10, lunch_duration=30):
    schedule = []
    current_datetime = start_datetime

    # Convert variety_shift_lengths DataFrame to a dictionary for faster lookups
    shift_length_dict = variety_shift_lengths.set_index('variety')['length'].to_dict()

    while not data.empty:
        for i, row in data.iterrows():
            variety = row['variety']
            remaining_minutes = row['productive_minutes']

            # Get the shift length for this variety
            shift_length = shift_length_dict.get(variety, timedelta(minutes=600))  # Default to 600 minutes if not found

            while remaining_minutes > 0:
                effective_shift_time, total_break_time_per_shift = calculate_shift_time_with_breaks(
                    shift_length=shift_length, break_duration=break_duration, lunch_duration=lunch_duration)

                # Skip non-working days
                if not is_working_day(current_datetime):
                    current_datetime = next_working_day(current_datetime)

                # Determine whether the current time is for a day or night shift
                if current_datetime.time() == datetime.strptime('06:00', '%H:%M').time():
                    shift_start_time = current_datetime.replace(hour=6, minute=0)
                    shift_type = 'Day'
                elif current_datetime.time() == datetime.strptime('18:00', '%H:%M').time():
                    shift_start_time = current_datetime.replace(hour=18, minute=0)
                    shift_type = 'Night'
                else:
                    # Adjust the time to the next appropriate shift start
                    if current_datetime.time() < datetime.strptime('06:00', '%H:%M').time():
                        shift_start_time = current_datetime.replace(hour=6, minute=0)
                        shift_type = 'Day'
                    else:
                        shift_start_time = current_datetime.replace(hour=18, minute=0)
                        shift_type = 'Night'

                shift_end_time = shift_start_time + shift_length
                actual_shift_time = min(remaining_minutes, effective_shift_time.total_seconds() / 60)
                remaining_minutes -= actual_shift_time

                schedule.append({
                    'Variety': variety,
                    'ShiftStart': shift_start_time,
                    'ShiftEnd': shift_end_time,
                    'ShiftType': shift_type,
                    'ShiftTime': shift_length.total_seconds() / 60,  # Convert to minutes
                    'EffectiveShiftTime': actual_shift_time,
                    'RemainingMinutes': remaining_minutes,
                    'BreakTime': total_break_time_per_shift,
                    'LunchBreak': lunch_duration
                })

                current_datetime = shift_end_time

                # Alternate between day and night shifts without skipping days
                if shift_type == 'Day':
                    # Move to night shift on the same day
                    current_datetime = shift_start_time.replace(hour=18, minute=0)
                else:
                    # Move to the next working day after night shift
                    current_datetime = shift_start_time.replace(hour=6, minute=0) + timedelta(days=1)

            # Deduct the leftover from the next variety's remaining minutes
            if remaining_minutes < (effective_shift_time.total_seconds() / 60):  # Convert effective_shift_time to minutes
                leftover = (effective_shift_time.total_seconds() / 60) - remaining_minutes
                remaining_minutes = 0  # Set current variety's remaining minutes to 0

                # Deduct leftover from the next variety's remaining minutes
                if i + 1 < len(data):
                    data.at[i + 1, 'productive_minutes'] -= leftover

                # If remaining minutes for this variety are zero, remove it from the DataFrame
            if remaining_minutes <= 0:
                data = data.drop(i).reset_index(drop=True)
                break  # Move to the next variety in the next iteration

    return pd.DataFrame(schedule)

# Example usage
start_datetime = datetime(2024, 9, 1, 6, 0)

# Example variety_shift_lengths DataFrame
variety_shift_lengths = pd.DataFrame({
    'variety': ['Simcoe', 'Citra', 'Sabro', 'Mosaic', 'HBC 1019', 'Palisade', 'HBC 522', 'Idaho 7', 'Talus', 'HBC 682', 'HBC 638', 'HBC 586', 'Experimental'],
    
    'length': [timedelta(minutes=600), timedelta(minutes=600), timedelta(minutes=600), timedelta(minutes=600), timedelta(minutes=600), timedelta(minutes=480), timedelta(minutes=480), timedelta(minutes=480), timedelta(minutes=480), timedelta(minutes=480), timedelta(minutes=480), timedelta(minutes=480), timedelta(minutes=480)]
})

# Define the order in which varieties need to be picked
picking_order = [
    'Simcoe', 'Sabro', 'Citra', 'Mosaic', 'HBC 1019', 'Palisade', 'HBC 522', 'Idaho 7', 'Talus', 'HBC 682',
    'HBC 638', 'HBC 586', 'Experimental'
]

# Sort the DataFrame based on the picking order
perrault['order'] = perrault['variety'].apply(lambda x: picking_order.index(x))
perrault = perrault.sort_values(by='order', ascending=True)

# Assuming 'perrault' is your DataFrame with the appropriate structure
shift_schedule = create_shift_schedule_with_breaks(
    data=perrault,
    start_datetime=start_datetime,
    variety_shift_lengths=variety_shift_lengths
)

# Output the schedule
print(shift_schedule)


    Variety          ShiftStart            ShiftEnd ShiftType  ShiftTime  \
0    Simcoe 2024-09-02 06:00:00 2024-09-02 16:00:00       Day      600.0   
1    Simcoe 2024-09-02 18:00:00 2024-09-03 04:00:00     Night      600.0   
2    Simcoe 2024-09-03 06:00:00 2024-09-03 16:00:00       Day      600.0   
3    Simcoe 2024-09-03 18:00:00 2024-09-04 04:00:00     Night      600.0   
4    Simcoe 2024-09-04 06:00:00 2024-09-04 16:00:00       Day      600.0   
5    Simcoe 2024-09-04 18:00:00 2024-09-05 04:00:00     Night      600.0   
6    Simcoe 2024-09-05 06:00:00 2024-09-05 16:00:00       Day      600.0   
7    Simcoe 2024-09-05 18:00:00 2024-09-06 04:00:00     Night      600.0   
8    Simcoe 2024-09-06 06:00:00 2024-09-06 16:00:00       Day      600.0   
9    Simcoe 2024-09-06 18:00:00 2024-09-07 04:00:00     Night      600.0   
10   Simcoe 2024-09-07 06:00:00 2024-09-07 16:00:00       Day      600.0   
11   Simcoe 2024-09-07 18:00:00 2024-09-08 04:00:00     Night      600.0   
12   Simcoe 

In [33]:
shift_schedule

,Variety,ShiftStart,ShiftEnd,ShiftType,ShiftTime,EffectiveShiftTime,RemainingMinutes,BreakTime,LunchBreak
0,Simcoe,2024-09-02 06:00:00,2024-09-02 16:00:00,Day,600.0,570.000000,8163.401646,20.0,30
1,Simcoe,2024-09-02 18:00:00,2024-09-03 04:00:00,Night,600.0,570.000000,7593.401646,20.0,30
2,Simcoe,2024-09-03 06:00:00,2024-09-03 16:00:00,Day,600.0,570.000000,7023.401646,20.0,30
3,Simcoe,2024-09-03 18:00:00,2024-09-04 04:00:00,Night,600.0,570.000000,6453.401646,20.0,30
4,Simcoe,2024-09-04 06:00:00,2024-09-04 16:00:00,Day,600.0,570.000000,5883.401646,20.0,30
5,Simcoe,2024-09-04 18:00:00,2024-09-05 04:00:00,Night,600.0,570.000000,5313.401646,20.0,30
6,Simcoe,2024-09-05 06:00:00,2024-09-05 16:00:00,Day,600.0,570.000000,4743.401646,20.0,30
7,Simcoe,2024-09-05 18:00:00,2024-09-06 04:00:00,Night,600.0,570.000000,4173.401646,20.0,30
8,Simcoe,2024-09-06 06:00:00,2024-09-06 16:00:00,Day,600.0,570.000000,3603.401646,20.0,30
9,Simcoe,2024-09-06 18:00:00,2024-09-07 04:00:00,Night,600.0,570.000000,3033.401646,20.0,30
